In [ ]:
from analysis.analysis import *
import matplotlib.pyplot as plt
import pandas as pd

# Useful Functions

In [4]:
def heatmap(
    ax,
    F,
    bounds,
    labels,
    end_pts,
    xlabel=False,
    ylabel=False,
    cbar=False,
    cbar_label=False,
    cmap=None,
):
    xmin, xmax, vmin, vmax, nbins = bounds
    xmin, xmax = np.array([xmin, xmax]) - (xmax + xmin) / 2  # center along x
    ax.set_xlabel(r"$x$ ($\mu$m)") if xlabel else ...
    ax.set_ylabel(r"$v$ ($\mu$m/hr)") if ylabel else ...
    _kwargs = {
        "extent": [xmin, xmax, vmin, vmax],
        "origin": "lower",
        "interpolation": "bilinear",
    }
    if cmap is None:
        cmap = "jet"
    im = ax.imshow(F, cmap=cmap, **_kwargs)
    if cbar:
        cbar = plt.colorbar(im, ax=ax)
        cbar.set_ticks([-1000, 0, 1000])
        cbar.formatter.set_powerlimits((0, 0))
        cbar.formatter.set_useMathText(True)
    cbar.set_label(r"$F$ ($\mu$m/hr$^2$)") if cbar_label else ...
    ax.set_aspect("auto")


def clean_init_pts(pts, labels, nuke):
    df = pd.DataFrame({"x": pts[:, 0], "y": pts[:, 1], "color": labels})
    if nuke:
        df = df.drop(df.query("y < 30 and y > -30 and color == 'red'").index)

    return df[df.y < 160].reset_index(drop=True)


def streamplot(
    F,
    bounds,
    labels,
    end_pts,
    ax,
    title="",
    n_skip=1,
    zero_line=False,
    show_terminal=None,
    septx=False,
    darker_shade=False,
    mix_corn_salmon=False,
    ignore_wheat=False,
    include_red=True,
    nuke_red_in_middle=True,
    bootstrap_flag=False,
    **kwargs,
):
    xmin, xmax, vmin, vmax, nbins = bounds
    x_c = (xmax + xmin) / 2
    xmin, xmax = np.array([xmin, xmax]) - x_c  # center along x
    ax.set_xlabel(r"$x$ ($\mu$m)")
    ax.set_ylabel(r"$v$ ($\mu$m/hr)")

    # drop labels where F is nan
    if bootstrap_flag:
        labels = labels[~np.isnan(F.ravel())]

    X, Y, init_pts = full_lattice(F, xmin, xmax, vmin, vmax, nbins)
    init_pts = clean_init_pts(init_pts, labels, nuke_red_in_middle)
    print(f"All unique colors: {init_pts.color.unique()}")

    for lbl, pts in init_pts.groupby("color"):
        if lbl == "gainsboro":
            continue

        if lbl == "red" and not include_red:
            continue

        _pts = pts[["x", "y"]].values

        if lbl != "wheat" or ignore_wheat:
            _pts = _pts[::n_skip]
            if mix_corn_salmon:
                lbl = "cornflowerblue"

        if lbl == "wheat":
            lbl = "goldenrod"
            if ignore_wheat:
                lbl = "cornflowerblue"

        if darker_shade:
            if lbl == "cornflowerblue":
                lbl = "royalblue"
            elif lbl == "salmon":
                lbl = "tomato"
            else:
                pass

        stream_kwargs = {"color": lbl, "start_points": _pts} | kwargs
        ax.streamplot(X, Y, Y, F, **stream_kwargs)
        ax.set_title(title)
        if zero_line:
            ax.axhline(y=0, lw=1, color="black", zorder=0)

        if show_terminal is not None:
            color = show_terminal.get("color", "black")
            s = show_terminal.get("size", 50)
            interior = show_terminal.get("interior", False)
            # mask = np.fabs(end_pts[:, 1]) < 10
            # end_pts = end_pts[mask]

            if interior:
                xy = (
                    pd.DataFrame({"x": end_pts[:, 0], "y": end_pts[:, 1]})
                    .query("x < 165 and x > 135 and y < 20 and y > -20")
                    .values
                )
                ax.scatter(
                    xy[:, 0] - x_c,
                    xy[:, 1],
                    s=s,
                    color=color,
                    marker="x",
                    zorder=10,
                )
            else:
                ax.scatter(
                    end_pts[:, 0] - x_c,
                    end_pts[:, 1],
                    s=s,
                    color=color,
                    marker="x",
                    zorder=10,
                )

    if septx:
        xmin, xmax, vmin, vmax, nbins = bounds
        _, _, init_pts = full_lattice(F, xmin, xmax, vmin, vmax, nbins)
        init_pts_df = clean_init_pts(init_pts, labels, nuke_red_in_middle)
        init_pts = init_pts_df[["x", "y"]].values
        labels = init_pts_df.color.values

        img = lattice_to_image(init_pts, labels, bounds)
        if not mix_corn_salmon:
            sepX_1 = get_separatrices(
                img, levels=[1.5], origin="lower", extent=[xmin, xmax, vmin, vmax]
            )
            ax.plot(sepX_1[:, 0] - x_c, sepX_1[:, 1], lw=2, color="black")

        if np.any(np.where(img == 1)):
            arr = np.where(img == 1, 1, 0)  # cluster `wheat`
            sepX_2 = get_separatrices(
                arr, levels=[0.5], origin="lower", extent=[xmin, xmax, vmin, vmax]
            )
            ax.plot(sepX_2[:, 0] - x_c, sepX_2[:, 1], lw=2, color="black")


def dynamicsplot(ax, F, bounds, labels, end_pts, delta=1, **kwargs):
    X, Y, init_pts = full_lattice(F, *bounds)

    # raw
    v, f = evaluate_F_v_x0(Y, F, bounds[-1], delta=delta)
    lim = np.fabs([v.min(), v.max()]).min()
    v_interp = np.linspace(-lim, lim, 30)
    f_interp = np.interp(v_interp, v, f)

    kwargs = {"lw": 2, "color": "black"} | kwargs
    ax.plot(v_interp, f_interp, "-o", **kwargs)
    ax.grid(which="major")
    ax.set_xlabel(r"$v$ ($\mu$m/hr)")
    ax.set_ylabel(r"$F(x\rightarrow 0, v)$ ($\mu$m/hr$^2$)")


def process_gid(gid):
    import pickle

    data = pickle.load(open(f"maps/map_{gid}.pkl", "rb"))
    if len(data["labels"]) > 0:
        return data["F"], data["bounds"], np.array(data["labels"]), data["end_pts"]
    return data["F"], data["bounds"]

In [31]:
def read_bootstrap_map(gid, k):
    import pickle

    data_root = f"data/bootstrapped_maps_full"
    data = pickle.load(open(f"{data_root}/map_{gid}_{k}.pkl", "rb"))
    return (
        data["F"],
        data["bounds"],
        np.array(data["labels"]),
        data["end_pts"],
    )


def get_unique_states_from_bootstraps(gid, mix_salmon_corn, ignore_wheat):
    from glob import glob
    import pickle

    files = glob(f"data/bootstrapped_maps_full/map_{gid}*")
    labels = [pd.Series(pickle.load(open(f, "rb"))["labels"]) for f in files]
    _data = pickle.load(open(files[0], "rb"))
    F, bounds = _data["F"], _data["bounds"]
    labels = pd.concat(labels, axis=1)
    if mix_salmon_corn:
        labels.replace("salmon", "cornflowerblue", inplace=True)
    if ignore_wheat:
        labels.replace("wheat", np.nan, inplace=True)
    labels = (
        labels.replace(["red", "gainsboro"], np.nan)
        .nunique(axis=1, dropna=True)
        .replace(0, np.nan)
    )
    return F, bounds, labels


def bootstrapped_state_space(
    gid, mean_septx=True, mix_salmon_corn=False, ignore_wheat=False, **kwargs
):
    F, bounds, flipped_counts = get_unique_states_from_bootstraps(
        gid, mix_salmon_corn, ignore_wheat
    )
    xmin, xmax, vmin, vmax, nbins = bounds
    dx = (xmax - xmin) / nbins
    dv = (vmax - vmin) / nbins
    X, Y = np.meshgrid(np.arange(nbins), np.arange(nbins))
    X = X * dx + xmin + dx / 2 - (xmax + xmin) / 2
    Y = Y * dv + vmin + dv / 2
    init_pts = np.hstack([X.reshape(-1, 1), Y.reshape(-1, 1)])

    figsize = kwargs.get("figsize", (3, 3))
    cmap = kwargs.get("cmap", "Purples")
    size = kwargs.get("size", 5)
    plt.figure(figsize=figsize)
    plt.scatter(init_pts[:, 0], init_pts[:, 1], c=flipped_counts, cmap=cmap, s=size)
    plt.xlabel(r"$x$ ($\mu$m)")
    plt.ylabel(r"$v$ ($\mu$m/hr)")
    cbar = plt.colorbar(
        ticks=np.arange(flipped_counts.max() + 1), orientation="horizontal", pad=0.2
    )
    cbar.set_label("num of phases")

    if mean_septx:
        F, bounds, labels, end_pts = process_gid(gid)
        xmin, xmax, vmin, vmax, nbins = bounds
        _, _, init_pts = full_lattice(F, xmin, xmax, vmin, vmax, nbins)
        init_pts_df = clean_init_pts(init_pts, labels, nuke=True)
        init_pts = init_pts_df[["x", "y"]].values
        labels = init_pts_df.color.values

        img = lattice_to_image(init_pts, labels, bounds)
        if not mix_salmon_corn:
            sepX_1 = get_separatrices(
                img, levels=[1.5], origin="lower", extent=[xmin, xmax, vmin, vmax]
            )
            plt.plot(
                sepX_1[:, 0] - (xmax + xmin) / 2, sepX_1[:, 1], lw=2, color="black"
            )

        if np.any(np.where(img == 1)):
            arr = np.where(img == 1, 1, 0)  # cluster `wheat`
            sepX_2 = get_separatrices(
                arr, levels=[0.5], origin="lower", extent=[xmin, xmax, vmin, vmax]
            )
            plt.plot(
                sepX_2[:, 0] - (xmax + xmin) / 2, sepX_2[:, 1], lw=2, color="black"
            )

    return plt.gca()


def boot_ids(gid):
    from glob import glob

    data_root = f"data/bootstrapped_maps_clean"
    return np.array(
        [int(f.split("_")[-1].split(".")[0]) for f in glob(data_root + f"/map_{gid}*")]
    )

# Global Plotting Metadata

In [6]:
kwargs = {
    "density": 10,
    "linewidth": 0.5,
    "arrowsize": 1.0,
    "integration_direction": "forward",
    "broken_streamlines": True,
}
nskip = 2
inline = True
render = lambda f, dpi=300: plt.show() if inline else plt.savefig(f"{f}.jpg", dpi=dpi)

---

# Generating Paper Figures

### Fig 1

In [ ]:
from substrate.substrates import Substrate
import matplotlib.pyplot as plt

sub_gen = Substrate(200, 50, xi=0.2)
rect = sub_gen.rectangular()

ext = 0
l = 38 + ext
d = 73 + ext
ts = sub_gen.two_state_sub(square_width=l, bridge_width=17, delta_centers=d)

# fig, ax = plt.subplot_mosaic([[0, 1], [2, 2]], figsize=(3, 2), dpi=300)
fig, ax = plt.subplots(1, 2, figsize=(4, 2), dpi=300)

for a, im, title in zip(ax, [ts, rect], ["Two-state", "Rectangular"]):
    alpha = 0 if title == "" else 1
    a.imshow(im, extent=[0, 50 * 6, 0, 50 * 6], cmap="Greys_r", alpha=alpha)
    # a.contour(im, levels=[0.5], linewidths=[1], colors=["cornflowerblue"], extent=[0, 50 * 6, 0, 50 * 6])
    a.set_xlim([80, 220])
    a.set_ylim([100, 200])
    a.get_xaxis().set_visible(False)
    a.get_yaxis().set_visible(False)
    a.set_frame_on(False)
    a.set_title(title, fontsize=10)

    if title == "Two-state":
        a.arrow(
            100,
            110,
            10,
            0,
            width=0.001,
            head_width=3,
            head_length=2,
            color="black",
        )
        a.arrow(
            100,
            110,
            0,
            10,
            width=0.001,
            head_width=3,
            head_length=2,
            color="black",
        )
        a.text(115, 110, r"$x$", {"fontsize": 8})
        a.text(105, 120, r"$y$", {"fontsize": 8})


from matplotlib_scalebar.scalebar import ScaleBar

scalebar = ScaleBar(
    dx=1,
    units="um",
    location="center left",
    pad=-2,
    border_pad=0,
    box_alpha=0,
    font_properties={"size": 8},
)
a.add_artist(scalebar)

plt.savefig("paper_plots/geometries.jpg")
plt.show()

### Fig 3(c, g, k)
Time series plots and c.m. trajectories were made server-side

In [ ]:
nrows, ncols = 1, 1
fig, axs = plt.subplots(nrows, ncols, figsize=(3.5 * ncols, 3.5 * nrows))
heatmap(axs, *process_gid(10), xlabel=True, cbar=True, cbar_label=True)
streamplot(
    *process_gid(10),
    axs,
    n_skip=nskip,
    zero_line=True,
    # show_terminal=True,
    **kwargs | {"color": "black"},
)
render("paper_plots/limit_cycle_defaults")

nrows, ncols = 1, 1
fig, axs = plt.subplots(nrows, ncols, figsize=(3.5 * ncols, 3.5 * nrows))
heatmap(axs, *process_gid(26), xlabel=True, cbar=True, cbar_label=True)
streamplot(
    *process_gid(26),
    axs,
    n_skip=nskip,
    zero_line=True,
    ignore_wheat=True,
    # show_terminal={"color": "red"},
    **kwargs | {"color": "black"},
)
render(f"paper_plots/stationary_defaults")

nrows, ncols = 1, 1
fig, axs = plt.subplots(nrows, ncols, figsize=(3.5 * ncols, 3.5 * nrows))
heatmap(axs, *process_gid(108), xlabel=True, cbar=True, cbar_label=True)
streamplot(
    *process_gid(2),
    axs,
    n_skip=nskip,
    zero_line=True,
    show_terminal={"color": "black"},
    septx=True,
    darker_shade=True,
    **kwargs,
)
render(f"paper_plots/bistable_defaults")

### Fig. 5

In [ ]:
nrows, ncols = 1, 1
fig, axs = plt.subplots(nrows, ncols, figsize=(7 * ncols, 4 * nrows))
dynamicsplot(axs, *process_gid(151), color="black", label="free 2D")
dynamicsplot(axs, *process_gid(26), color="red", label="rectangular\n(stationary)")
dynamicsplot(axs, *process_gid(10), color="blue", label="two-state\n(limit-cycle)")
dynamicsplot(axs, *process_gid(2), color="limegreen", label="two-state\n(bistable)")
axs.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
plt.legend(bbox_to_anchor=(1, 1))
plt.tight_layout()
render("paper_plots/nonlinear_plots")

### Streamlines as cell size changes

In [ ]:
nrows, ncols = 1, 3
fig, axs = plt.subplots(nrows, ncols, figsize=(2.5 * ncols, 3.5 * nrows), sharey=True)

streamplot(
    *process_gid(2),
    axs[0],
    title=r"$R_0=15\mu\mathrm{m}$",
    n_skip=nskip,
    zero_line=True,
    show_terminal={"color": "black"},
    septx=True,
    **kwargs
)
streamplot(
    *process_gid(10),
    axs[1],
    title=r"$R_0=18\mu\mathrm{m}$",
    n_skip=nskip,
    zero_line=True,
    **kwargs | {"color": "cornflowerblue"}
)
streamplot(
    *process_gid(122),
    axs[2],
    title=r"$R_0=21\mu\mathrm{m}$",
    n_skip=nskip,
    zero_line=True,
    **kwargs | {"color": "cornflowerblue"}
)
[axs[k].set_ylabel("") for k in range(1, 3)]

plt.tight_layout()
render("paper_plots/varying_cell_size_streams")

### Fig 4

In [ ]:
from matplotlib import colors


nrows, ncols = 1, 2
fig, axs = plt.subplots(nrows, ncols, figsize=(2.5 * ncols, 2.5 * nrows), sharey=True)

streamplot(
    *process_gid(10),
    axs[0],
    title=r"$\gamma=0.8\gamma_0$",
    n_skip=nskip,
    zero_line=True,
    mix_corn_salmon=True,
    # include_red=False,
    **kwargs
)
streamplot(
    *process_gid(110),
    axs[1],
    title=r"$\gamma=1.8\gamma_0$",
    n_skip=nskip,
    zero_line=True,
    mix_corn_salmon=True,
    septx=True,
    # include_red=True,
    **kwargs
)
axs[0].text(
    -110,
    0,
    r"$\tau_f=2.2$s",
    rotation=90,
    ha="center",
    va="center",
    bbox=dict(
        boxstyle="round",
        ec=colors.to_rgb("mediumorchid"),
        fc=colors.to_rgb("lavenderblush"),
    ),
)
axs[1].set_ylabel("")
axs[0].set_xlabel("")
axs[1].set_xlabel("")
axs[0].tick_params(labelbottom=False)
axs[1].tick_params(labelbottom=False)
plt.subplots_adjust(wspace=0.2)
render("paper_plots/varying_cell_tension_streams_A")

In [ ]:
nrows, ncols = 1, 2
fig, axs = plt.subplots(nrows, ncols, figsize=(2.5 * ncols, 2.5 * nrows), sharey=True)


streamplot(
    *process_gid(145),
    axs[0],
    n_skip=nskip,
    zero_line=True,
    mix_corn_salmon=True,
    ignore_wheat=True,
    nuke_red_in_middle=False,
    **kwargs
)
streamplot(
    *process_gid(108),
    axs[1],
    n_skip=nskip,
    zero_line=True,
    show_terminal={"color": "black"},
    septx=True,
    **kwargs
)
axs[1].set_ylabel("")
axs[0].text(
    -110,
    0,
    r"$\tau_f=3.6$s",
    rotation=90,
    ha="center",
    va="center",
    bbox=dict(
        boxstyle="round",
        ec=colors.to_rgb("mediumorchid"),
        fc=colors.to_rgb("lavenderblush"),
    ),
)

plt.subplots_adjust(wspace=0.2)
render("paper_plots/varying_cell_tension_streams_B")

### Fig 6

In [ ]:
nrows, ncols = 1, 7
fig, axs = plt.subplots(nrows, ncols, figsize=(3 * ncols, 3.5 * nrows))

gids = [132, 10, 133, 10, 136, 26, 137]
titles = [
    r"$\mu_{\beta}=.63\mathrm{min}^{-1}$",
    r"$\mu_{\beta}=.94\mathrm{min}^{-1}$",
    r"$\mu_{\beta}=1.4\mathrm{min}^{-1}$",
]
# streams
for ax, gid, title in zip(axs[:3], gids[:3], titles):
    streamplot(
        *process_gid(gid),
        ax,
        title=title,
        n_skip=nskip,
        zero_line=True,
        mix_corn_salmon=True,
        **kwargs
    )

for ax, gid in zip(axs[4:], gids[4:]):
    streamplot(
        *process_gid(gid),
        ax,
        n_skip=nskip,
        zero_line=True,
        mix_corn_salmon=True,
        # show_terminal=True,
        ignore_wheat=True,
        **kwargs | {"color": "cornflowerblue"}
    )

axs[0].clear()
streamplot(
    *process_gid(gids[0]),
    axs[0],
    title=titles[0],
    n_skip=nskip,
    zero_line=True,
    mix_corn_salmon=True,
    ignore_wheat=True,
    **kwargs
)

axs[2].clear()
streamplot(
    *process_gid(gids[2]),
    axs[2],
    title=titles[2],
    n_skip=nskip,
    zero_line=True,
    mix_corn_salmon=True,
    septx=True,
    **kwargs
)

axs[3].remove()
[axs[k].set_ylabel("") for k in range(1, ncols)]
[axs[k].sharey(axs[0]) for k in [1, 2]]
[axs[k].tick_params(labelleft=False) for k in [1, 2]]
[axs[k].sharey(axs[-1]) for k in [4, 5, 6]]
[axs[k].tick_params(labelleft=False) for k in [5, 6]]

plt.tight_layout()
render("paper_plots/varying_mub_streams")

In [ ]:
nrows, ncols = 1, 7
fig, axs = plt.subplots(nrows, ncols, figsize=(3 * ncols, 3.5 * nrows))

gids = [112, 10, 114, 10, 113, 26, 115]
titles = [
    r"$\sigma_{\beta}=2.8\mathrm{min}^{-1}$",
    r"$\sigma_{\beta}=4.7\mathrm{min}^{-1}$",
    r"$\sigma_{\beta}=6.6\mathrm{min}^{-1}$",
]

# streams
for ax, gid, title in zip(axs[:3], gids[:3], titles):
    streamplot(
        *process_gid(gid),
        ax,
        title=title,
        n_skip=nskip,
        zero_line=True,
        mix_corn_salmon=True,
        **kwargs
    )

for ax, gid in zip(axs[4:], gids[4:]):
    streamplot(
        *process_gid(gid),
        ax,
        n_skip=nskip,
        zero_line=True,
        mix_corn_salmon=True,
        # show_terminal=True,
        ignore_wheat=True,
        **kwargs | {"color": "cornflowerblue"}
    )

axs[2].clear()
streamplot(
    *process_gid(gids[2]),
    axs[2],
    title=titles[2],
    n_skip=nskip,
    zero_line=True,
    mix_corn_salmon=True,
    ignore_wheat=True,
    **kwargs
)

axs[0].clear()
streamplot(
    *process_gid(gids[0]),
    axs[0],
    title=titles[0],
    n_skip=nskip,
    zero_line=True,
    mix_corn_salmon=True,
    septx=True,
    show_terminal={"color": "black", "interior": True},
    **kwargs
)

axs[3].remove()
[axs[k].set_ylabel("") for k in range(1, ncols)]
[axs[k].sharey(axs[0]) for k in [1, 2]]
[axs[k].tick_params(labelleft=False) for k in [1, 2]]
[axs[k].sharey(axs[-1]) for k in [4, 5, 6]]
[axs[k].tick_params(labelleft=False) for k in [5, 6]]

plt.tight_layout()
render("paper_plots/varying_sigmab_streams")

In [ ]:
nrows, ncols = 1, 7
fig, axs = plt.subplots(nrows, ncols, figsize=(3 * ncols, 3.5 * nrows))

gids = [139, 10, 141, 10, 142, 26, 144]
titles = [
    r"$\sigma=3.4\mu \mathrm{m}$",
    r"$\sigma=6.7\mu \mathrm{m}$",
    r"$\sigma=12\mu \mathrm{m}$",
]

# streams
for ax, gid, title in zip(axs[:3], gids[:3], titles):
    streamplot(
        *process_gid(gid),
        ax,
        title=title,
        n_skip=nskip,
        zero_line=True,
        mix_corn_salmon=True,
        **kwargs
    )

for ax, gid in zip(axs[4:], gids[4:]):
    streamplot(
        *process_gid(gid),
        ax,
        n_skip=nskip,
        zero_line=True,
        mix_corn_salmon=True,
        # show_terminal=True,
        ignore_wheat=True,
        **kwargs | {"color": "cornflowerblue"}
    )

axs[0].clear()
axs[2].clear()
axs[4].clear()

streamplot(
    *process_gid(gids[0]),
    axs[0],
    n_skip=nskip,
    title=titles[0],
    zero_line=True,
    mix_corn_salmon=True,
    ignore_wheat=True,
    nuke_red_in_middle=False,
    # show_terminal={"color": "black"},
    **kwargs
)
streamplot(
    *process_gid(gids[2]),
    axs[2],
    title=titles[2],
    n_skip=nskip,
    zero_line=True,
    # mix_corn_salmon=True,
    show_terminal={"color": "black"},
    septx=True,
    **kwargs
)

streamplot(
    *process_gid(gids[4]),
    axs[4],
    n_skip=nskip,
    zero_line=True,
    septx=True,
    # mix_corn_salmon=True,
    show_terminal={"color": "black"},
    **kwargs
)

axs[3].remove()
[axs[k].set_ylabel("") for k in range(1, ncols)]
[axs[k].sharey(axs[0]) for k in [1, 2]]
[axs[k].tick_params(labelleft=False) for k in [1, 2]]
[axs[k].sharey(axs[-1]) for k in [4, 5, 6]]
[axs[k].tick_params(labelleft=False) for k in [5, 6]]

plt.tight_layout()
render("paper_plots/varying_sigma_streams")

In [ ]:
nrows, ncols = 1, 7
fig, axs = plt.subplots(nrows, ncols, figsize=(3 * ncols, 3.5 * nrows))

gids = [10, 145, 120, 10, 26, 146, 121]
titles = [
    r"$\tau_{f}=2.2\mathrm{s}$",
    r"$\tau_{f}=3.6\mathrm{s}$",
    r"$\tau_{f}=5.1\mathrm{s}$",
]

# streams
for ax, gid, title in zip(axs[:3], gids[:3], titles):
    streamplot(
        *process_gid(gid),
        ax,
        title=title,
        n_skip=nskip,
        zero_line=True,
        mix_corn_salmon=True,
        ignore_wheat=True,
        nuke_red_in_middle=False,
        **kwargs
    )

for ax, gid in zip(axs[4:], gids[4:]):
    streamplot(
        *process_gid(gid),
        ax,
        n_skip=nskip,
        zero_line=True,
        mix_corn_salmon=True,
        # show_terminal=True,
        ignore_wheat=True,
        **kwargs | {"color": "cornflowerblue"}
    )


axs[3].remove()
[axs[k].set_ylabel("") for k in range(1, ncols)]
[axs[k].sharey(axs[0]) for k in [1, 2]]
[axs[k].tick_params(labelleft=False) for k in [1, 2]]
[axs[k].sharey(axs[-1]) for k in [4, 5, 6]]
[axs[k].tick_params(labelleft=False) for k in [5, 6]]

plt.tight_layout()
# render("paper_plots/varying_tauf_streams")
render("paper_plots/varying_tauf_streams")

### Fig. S2

In [ ]:
nrows, ncols = 1, 2
fig, axs = plt.subplots(nrows, ncols, figsize=(4 * ncols, 3.5 * nrows), sharey=False)

heatmap(axs[0], *process_gid(100), xlabel=True, cbar=True, cbar_label=True)
streamplot(
    *process_gid(100),
    axs[0],
    n_skip=nskip,
    zero_line=True,
    mix_corn_salmon=True,
    **(kwargs | {"color": "black"})
)

dynamicsplot(axs[1], *process_gid(100), delta=2)
plt.tight_layout()
render("paper_plots/filopodia_knx")

# ===

nrows, ncols = 1, 2
fig, axs = plt.subplots(nrows, ncols, figsize=(4 * ncols, 3.5 * nrows), sharey=False)

heatmap(axs[0], *process_gid(101), xlabel=True, cbar=True, cbar_label=True)
streamplot(
    *process_gid(101),
    axs[0],
    n_skip=nskip,
    zero_line=True,
    mix_corn_salmon=True,
    ignore_wheat=True,
    **(kwargs | {"color": "black"})
)

dynamicsplot(axs[1], *process_gid(101))
plt.tight_layout()
render("paper_plots/filopodia_knx_rectangular")

### Fig. 7

In [ ]:
nrows, ncols = 1, 1
fig, axs = plt.subplots(nrows, ncols, figsize=(3.5 * ncols, 2 * nrows))
hist_kwargs = {"density": True, "bins": 30, "alpha": 0.75}
cs = ["purple", "orange"]
lbs = ["default model", "filopodia knockout"]
for k, x in enumerate(
    [
        np.load("data/fulltake_gid10.npy"),
        np.load("data/fulltake_gid100.npy"),
    ]
):
    axs.hist(x, **(hist_kwargs | {"color": cs[k], "label": lbs[k]}))
axs.set_ylabel("density")
axs.set_xlabel(r"$t_{\mathrm{hopping}}\ [\mathrm{hr}]$")
axs.legend(bbox_to_anchor=(1, 1))
plt.tight_layout()
render("paper_plots/filopodia_knx_hopping_times")

### Fig. 8(b) and S3

In [ ]:
nrows, ncols = 1, 2
fig, axs = plt.subplots(nrows, ncols, figsize=(4 * ncols, 3.5 * nrows))

heatmap(axs[0], *process_gid(12), xlabel=True, cbar=True, cbar_label=True)
streamplot(
    *process_gid(12),
    axs[0],
    n_skip=nskip,
    zero_line=True,
    darker_shade=True,
    septx=True,
    show_terminal={"color": "black"},
    **kwargs
)
dynamicsplot(axs[1], *process_gid(12), delta=3)

plt.tight_layout()
render("paper_plots/tauX_knx")

# ===

nrows, ncols = 1, 2
fig, axs = plt.subplots(nrows, ncols, figsize=(4 * ncols, 3.5 * nrows))

heatmap(axs[0], *process_gid(149), xlabel=True, cbar=True, cbar_label=True)
streamplot(
    *process_gid(149),
    axs[0],
    n_skip=nskip,
    zero_line=True,
    darker_shade=True,
    septx=False,
    mix_corn_salmon=True,
    ignore_wheat=True,
    # show_terminal={"color": "red", "size": 10},
    **kwargs | {"color": "black"}
)
dynamicsplot(axs[1], *process_gid(149))

plt.tight_layout()
render("paper_plots/tauX_knx_rectangular")

### Fig. S4

In [ ]:
nrows, ncols = 1, 2
fig, axs = plt.subplots(nrows, ncols, figsize=(4 * ncols, 3.5 * nrows))

heatmap(axs[0], *process_gid(13), xlabel=True, cbar=True, cbar_label=True)
streamplot(
    *process_gid(13),
    axs[0],
    n_skip=nskip,
    zero_line=True,
    **(kwargs | {"color": "black"})
)
dynamicsplot(axs[1], *process_gid(13))

plt.tight_layout()
render("paper_plots/tension_knx")

# ===

nrows, ncols = 1, 2
fig, axs = plt.subplots(nrows, ncols, figsize=(4 * ncols, 3.5 * nrows))

heatmap(axs[0], *process_gid(150), xlabel=True, cbar=True, cbar_label=True)
streamplot(
    *process_gid(150),
    axs[0],
    n_skip=nskip,
    zero_line=True,
    ignore_wheat=True,
    # show_terminal={"color": "red"},
    **(kwargs | {"color": "black"})
)
dynamicsplot(axs[1], *process_gid(150))

plt.tight_layout()
render("paper_plots/tension_knx_rectangular")

---
# Bootstraps

State space plots -- dot plots with each dot's categorical status over all bootstraps

In [ ]:
ignore_wheat = False
for gid, flag in zip(
    [2, 12, 108, 110, 112, 133, 141, 142],
    [False, False, False, True, True, True, False, False],
):
    if gid == 142:
        ignore_wheat = True

    bootstrapped_state_space(
        gid,
        mix_salmon_corn=flag,
        ignore_wheat=ignore_wheat,
        figsize=(3.5, 5.5),
    )
    render(f"paper_plots/_appendix/state_space_{gid}")

In [ ]:
ignore_wheat = False
for gid, flag in zip([2, 12, 108], [False, False, False]):
    fig, ax = plt.subplots(1, 1, figsize=(3, 3))
    streamplot(
        *process_gid(gid),
        ax,
        mix_corn_salmon=flag,
        ignore_wheat=ignore_wheat,
        septx=True,
        n_skip=nskip,
        zero_line=True,
        show_terminal={"color": "black"},
        **kwargs,
    )
    render(f"paper_plots/_appendix/main_{gid}")

gid = 112
ignore_wheat = False
fig, ax = plt.subplots(1, 1, figsize=(3, 3))
streamplot(
    *process_gid(gid),
    ax,
    mix_corn_salmon=True,
    ignore_wheat=ignore_wheat,
    septx=True,
    n_skip=nskip,
    zero_line=True,
    show_terminal={"color": "black", "interior": True},
    **kwargs,
)
render(f"paper_plots/_appendix/main_{gid}")

gid = 141
ignore_wheat = False
fig, ax = plt.subplots(1, 1, figsize=(3, 3))
streamplot(
    *process_gid(gid),
    ax,
    mix_corn_salmon=False,
    ignore_wheat=ignore_wheat,
    septx=True,
    n_skip=nskip,
    zero_line=True,
    show_terminal={"color": "black"},
    **kwargs,
)
render(f"paper_plots/_appendix/main_{gid}")

gid = 142
ignore_wheat = False
fig, ax = plt.subplots(1, 1, figsize=(3, 3))
streamplot(
    *process_gid(gid),
    ax,
    mix_corn_salmon=False,
    ignore_wheat=ignore_wheat,
    septx=True,
    n_skip=nskip,
    zero_line=True,
    show_terminal={"color": "black"},
    **kwargs,
)
render(f"paper_plots/_appendix/main_{gid}")

In [ ]:
ignore_wheat = False
for gid in [110, 133]:

    fig, ax = plt.subplots(1, 1, figsize=(3, 3))
    streamplot(
        *process_gid(gid),
        ax,
        mix_corn_salmon=True,
        ignore_wheat=ignore_wheat,
        septx=True,
        n_skip=nskip,
        zero_line=True,
        **kwargs,
    )
    render(f"paper_plots/_appendix/main_{gid}")

Streamplots from bootstraps

In [ ]:
n = 3
gid = 2
fig, axs = plt.subplots(n, n, figsize=(2.5 * n, 2.5 * n), sharex=True, sharey=True)
ks = boot_ids(gid)[
    np.random.choice(range(boot_ids(gid).size), size=n**2, replace=False)
]

for i, (ax, k) in enumerate(zip(axs.ravel(), ks)):
    streamplot(
        *read_bootstrap_map(gid, k),
        ax,
        n_skip=2,
        zero_line=True,
        show_terminal={"color": "black"},
        septx=True,
        bootstrap_flag=True,
        include_red=False,
        ignore_wheat=True,
        **kwargs,
    )
    ax.set_xlabel("")
    ax.set_ylabel("")
    # ax.set_title(f"{k}")

    if i % n == 0:
        ax.set_ylabel(r"$v$ ($\mu$m/hr)")
    if i >= n**2 - n:
        ax.set_xlabel(r"$x$ ($\mu$m)")

print(ks)
plt.tight_layout()
render(f"paper_plots/_appendix/boots_{gid}")

In [ ]:
n = 3
gid = 12
fig, axs = plt.subplots(n, n, figsize=(2.5 * n, 2.5 * n), sharex=True, sharey=True)
ks = boot_ids(gid)[
    np.random.choice(range(boot_ids(gid).size), size=n**2, replace=False)
]

for i, (ax, k) in enumerate(zip(axs.ravel(), ks)):
    streamplot(
        *read_bootstrap_map(gid, k),
        ax,
        n_skip=2,
        zero_line=True,
        show_terminal={"color": "black"},
        septx=True,
        bootstrap_flag=True,
        include_red=False,
        ignore_wheat=False,
        **kwargs,
    )
    ax.set_xlabel("")
    ax.set_ylabel("")
    # ax.set_title(f"{k}")

    if i % n == 0:
        ax.set_ylabel(r"$v$ ($\mu$m/hr)")
    if i >= n**2 - n:
        ax.set_xlabel(r"$x$ ($\mu$m)")

print(ks)
plt.tight_layout()
render(f"paper_plots/_appendix/boots_{gid}")

In [ ]:
n = 3
gid = 108
fig, axs = plt.subplots(n, n, figsize=(2.5 * n, 2.5 * n), sharex=True, sharey=True)
ks = boot_ids(gid)[
    np.random.choice(range(boot_ids(gid).size), size=n**2, replace=False)
]

for i, (ax, k) in enumerate(zip(axs.ravel(), ks)):
    streamplot(
        *read_bootstrap_map(gid, k),
        ax,
        n_skip=2,
        zero_line=True,
        show_terminal={"color": "black"},
        septx=True,
        bootstrap_flag=True,
        include_red=False,
        # mix_corn_salmon=True,
        **kwargs,
    )
    ax.set_xlabel("")
    ax.set_ylabel("")
    # ax.set_title(f"{k}")

    if i % n == 0:
        ax.set_ylabel(r"$v$ ($\mu$m/hr)")
    if i >= n**2 - n:
        ax.set_xlabel(r"$x$ ($\mu$m)")

print(ks)
plt.tight_layout()
render(f"paper_plots/_appendix/boots_{gid}")

In [ ]:
n = 3
gid = 110
fig, axs = plt.subplots(n, n, figsize=(2.5 * n, 2.5 * n), sharex=True, sharey=True)
ks = boot_ids(gid)[
    np.random.choice(range(boot_ids(gid).size), size=n**2, replace=False)
]

for i, (ax, k) in enumerate(zip(axs.ravel(), ks)):
    terminal = None
    if k not in [4, 10, 16, 22, 35]:
        terminal = {"color": "black", "interior": True}

    streamplot(
        *read_bootstrap_map(gid, k),
        ax,
        n_skip=2,
        zero_line=True,
        show_terminal=terminal,
        septx=True,
        bootstrap_flag=True,
        mix_corn_salmon=True,
        **kwargs,
    )
    ax.set_xlabel("")
    ax.set_ylabel("")
    # ax.set_title(f"{k}")

    if i % n == 0:
        ax.set_ylabel(r"$v$ ($\mu$m/hr)")
    if i >= n**2 - n:
        ax.set_xlabel(r"$x$ ($\mu$m)")

print(ks)
plt.tight_layout()
render(f"paper_plots/_appendix/boots_{gid}")

In [ ]:
n = 3
gid = 112
fig, axs = plt.subplots(n, n, figsize=(2.5 * n, 2.5 * n), sharex=True, sharey=True)
ks = boot_ids(gid)[
    np.random.choice(range(boot_ids(gid).size), size=n**2, replace=False)
]

for i, (ax, k) in enumerate(zip(axs.ravel(), ks)):
    streamplot(
        *read_bootstrap_map(gid, k),
        ax,
        n_skip=2,
        zero_line=True,
        show_terminal={"color": "black", "interior": True},
        septx=True,
        bootstrap_flag=True,
        mix_corn_salmon=True,
        **kwargs,
    )
    ax.set_xlabel("")
    ax.set_ylabel("")
    # ax.set_title(f"{k}")

    if i % n == 0:
        ax.set_ylabel(r"$v$ ($\mu$m/hr)")
    if i >= n**2 - n:
        ax.set_xlabel(r"$x$ ($\mu$m)")

print(ks)
plt.tight_layout()
render(f"paper_plots/_appendix/boots_{gid}")

In [ ]:
n = 3
gid = 133
fig, axs = plt.subplots(n, n, figsize=(2.5 * n, 2.5 * n), sharex=True, sharey=True)
ks = boot_ids(gid)[
    np.random.choice(range(boot_ids(gid).size), size=n**2, replace=False)
]

for i, (ax, k) in enumerate(zip(axs.ravel(), ks)):
    terminal = None
    if k not in [46, 22, 43]:
        terminal = {"color": "black", "interior": True}

    streamplot(
        *read_bootstrap_map(gid, k),
        ax,
        n_skip=2,
        zero_line=True,
        show_terminal=terminal,
        septx=True,
        bootstrap_flag=True,
        mix_corn_salmon=True,
        **kwargs,
    )
    ax.set_xlabel("")
    ax.set_ylabel("")
    # ax.set_title(f"{k}")

    if i % n == 0:
        ax.set_ylabel(r"$v$ ($\mu$m/hr)")
    if i >= n**2 - n:
        ax.set_xlabel(r"$x$ ($\mu$m)")

print(ks)
plt.tight_layout()
render(f"paper_plots/_appendix/boots_{gid}")

In [ ]:
n = 3
gid = 141
fig, axs = plt.subplots(n, n, figsize=(2.5 * n, 2.5 * n), sharex=True, sharey=True)
ks = boot_ids(gid)[
    np.random.choice(range(boot_ids(gid).size), size=n**2, replace=False)
]

for i, (ax, k) in enumerate(zip(axs.ravel(), ks)):
    streamplot(
        *read_bootstrap_map(gid, k),
        ax,
        n_skip=2,
        zero_line=True,
        show_terminal={"color": "black"},
        septx=True,
        bootstrap_flag=True,
        **kwargs,
    )
    ax.set_xlabel("")
    ax.set_ylabel("")
    # ax.set_title(f"{k}")

    if i % n == 0:
        ax.set_ylabel(r"$v$ ($\mu$m/hr)")
    if i >= n**2 - n:
        ax.set_xlabel(r"$x$ ($\mu$m)")

print(ks)
plt.tight_layout()
render(f"paper_plots/_appendix/boots_{gid}")

In [ ]:
n = 3
gid = 142
fig, axs = plt.subplots(n, n, figsize=(2.5 * n, 2.5 * n), sharex=True, sharey=True)
ks = boot_ids(gid)[
    np.random.choice(range(boot_ids(gid).size), size=n**2, replace=False)
]

for i, (ax, k) in enumerate(zip(axs.ravel(), ks)):
    streamplot(
        *read_bootstrap_map(gid, k),
        ax,
        n_skip=2,
        zero_line=True,
        show_terminal={"color": "black"},
        septx=True,
        bootstrap_flag=True,
        ignore_wheat=True,
        **kwargs,
    )
    ax.set_xlabel("")
    ax.set_ylabel("")

    if i % n == 0:
        ax.set_ylabel(r"$v$ ($\mu$m/hr)")
    if i >= n**2 - n:
        ax.set_xlabel(r"$x$ ($\mu$m)")

print(ks)
plt.tight_layout()
# render(f"paper_plots/_appendix/boots_{gid}")